# Example: Drift of Maximim Sharpe Ratio Portfolios
In this example, we will use historical data to compute the maximum Sharpe ratio portfolio, and the compute the drift of this portfolio over time. 

> __Learning Objectives:__
> 
> By the end of this example, you will be able to:
> Three key learning objectives for this example go here.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered daily open-high-low-close (OHLC) data for each firm in the [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2025` until `09-26-2025`, along with data for several exchange-traded funds and volatility products during that time period.

Let's load the `original_dataset::DataFrame` by calling [the `MyTestingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTestingMarketDataSet).

In [2]:
original_dataset = MyTestingMarketDataSet() |> x-> x["dataset"] # load the original dataset (testing)

Dict{String, DataFrame} with 482 entries:
  "NI"   => 182×8 DataFrame…
  "EMR"  => 182×8 DataFrame…
  "CTAS" => 182×8 DataFrame…
  "HSIC" => 182×8 DataFrame…
  "KIM"  => 182×8 DataFrame…
  "PLD"  => 182×8 DataFrame…
  "IEX"  => 182×8 DataFrame…
  "BAC"  => 182×8 DataFrame…
  "CBOE" => 182×8 DataFrame…
  "EXR"  => 182×8 DataFrame…
  "NCLH" => 182×8 DataFrame…
  "CVS"  => 182×8 DataFrame…
  "DRI"  => 182×8 DataFrame…
  "DTE"  => 182×8 DataFrame…
  "ZION" => 182×8 DataFrame…
  "AVY"  => 182×8 DataFrame…
  "EW"   => 182×8 DataFrame…
  "EA"   => 182×8 DataFrame…
  "NWSA" => 182×8 DataFrame…
  ⋮      => ⋮

Not all tickers in our dataset have the maximum number of trading days for various reasons, such as acquisition or delisting events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has the maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days::Int64` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow; # maximum number of trading days in our dataset

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # check if ticker has maximum trading days
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Let's get a list of the firms in our cleaned dataset and sort them alphabetically. We store the sorted firm ticker symbols in the `list_of_tickers::Array{String,1}` variable:

In [5]:
list_of_tickers = keys(dataset) |> collect |> sort;

Finally, let's load the single index model parameters that we computed in the previous example. We'll store this data in the `sim_model_parameters::Dict{String,NamedTuple}` variable:

In [6]:
sim_model_parameters, Ḡₘ, Varₘ = let

    # initialize -
    path_to_sim_model_parameters = joinpath(_PATH_TO_DATA,"SIMs-SPY-SP500-01-03-14-to-12-31-24.jld2");
    sim_model_parameters = JLD2.load(path_to_sim_model_parameters);
    parameters = sim_model_parameters["data"]; # return

    Ḡₘ = sim_model_parameters["Ḡₘ"]; # mean of market growth rates
    Varₘ = sim_model_parameters["Varₘ"]; # variance of market growth

    # return -
    parameters, Ḡₘ, Varₘ;
end;

In [7]:
Ḡₘ

0.10610026953971988

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, their units, and permissible values:

In [8]:
risk_free_rate = 0.0389; # hypothetical continuous compounded risk free rate (units: 1/year)
Δt = (1.0/252.0); # time step for 1-trading day (units: 1/year)

___

# Task 1: Compute the Maximum Sharpe Ratio Portfolio
In this task, we will compute the maximum Sharpe ratio portfolio using the single index model parameters we loaded earlier.

Let's start by specifying a collection of ticker symbols, e.g., `AAPL`, `MSFT`, etc., that interest you in the `my_list_of_tickers::Array{String,1}` array.

In [9]:
my_list_of_tickers = ["AAPL", "MSFT", "INTC", "MU", "AMD", "NVDA", "GS", "BAC", "WFC", 
    "C", "F", "GM", "JNJ"]; # random tickers that I selected.

### SIM-Based Risk and Return Matrices
Next, we'll compute the expected return vector and covariance matrix using the single index model. We'll use the SIM formula for expected returns and the SIM-derived covariance structure. Store these in the `μ̂_sim::Array{Float64,1}` and `Σ̂_sim::Array{Float64,2}` variables:

In [10]:
Ĝ_sim, Σ̂_sim = let

    # initialize -
    N = length(my_list_of_tickers); # number of assets in portfolio
    Ĝ_sim = Array{Float64,1}(); # drift vector
    Σ̂_sim = Array{Float64,2}(undef, N, N); # covariance matrix for *our* portfolio
    σ²ₘ = Varₘ; # variance of market factor (load from saved data, historical from 2014-2024)

    # compute the expected growth rate (return) for each of our tickers -
    for i ∈ eachindex(my_list_of_tickers)
        ticker = my_list_of_tickers[i];
        data = sim_model_parameters[ticker]; # get the data for this ticker
        αᵢ = data.alpha; # get alpha
        βᵢ = data.beta; # get beta
        Ḡᵢ = αᵢ + βᵢ* Ḡₘ; # compute the growth rate for this ticker
        push!(Ĝ_sim, Ḡᵢ); # append drift value to Ĝ_sim
    end

    # compute the covariance matrix using the single index model -
    for i ∈ eachindex(my_list_of_tickers)

        ticker_i = my_list_of_tickers[i];
        data_i = sim_model_parameters[ticker_i]; # get the data for ticker i
        βᵢ = data_i.beta; # get beta for ticker i
        σ²_εᵢ = (Δt)*data_i.training_variance; # residual variance for ticker i

        for j ∈ eachindex(my_list_of_tickers)
            
            ticker_j = my_list_of_tickers[j];
            data_j = sim_model_parameters[ticker_j]; # get the data for ticker j
            βⱼ = data_j.beta; # get beta for ticker j
            σ²_εⱼ = (Δt)*data_j.training_variance; # residual variance for ticker j
            
            if i == j
                Σ̂_sim[i,j] = βᵢ*βⱼ*σ²ₘ + σ²_εᵢ; # diagonal elements
            else
                Σ̂_sim[i,j] = βᵢ*βⱼ*σ²ₘ; # off-diagonal elements
            end
        end
    end

    (Ĝ_sim, Σ̂_sim*Δt); # return
end;

Fill me in later.

In [11]:
model = let

    # initialize - 
    N = length(my_list_of_tickers); # number of assets in portfolio
    α = zeros(N);
    β = zeros(N);

    # grab alphas and betas for our tickers -
    for i ∈ eachindex(my_list_of_tickers)
        ticker = my_list_of_tickers[i];
        data = sim_model_parameters[ticker]; # get the data for this ticker
        α[i] = data.alpha; # get alpha
        β[i] = data.beta; # get beta
    end

    # build the model
    model = build(MySharpeRatioPortfolioChoiceProblem, (
        Σ = Σ̂_sim,
        risk_free_rate = risk_free_rate,
        gₘ = Ḡₘ,
        α = α,
        β = β,
    ));

    model; # return -
end

MySharpeRatioPortfolioChoiceProblem([0.05500842390042998 0.025125810733577374 … 0.03205031438789042 0.011767893451041045; 0.025125810733577374 0.04643622315677927 … 0.03090811695047127 0.011348513547901024; … ; 0.03205031438789042 0.03090811695047127 … 0.1033028537736646 0.014476087195840908; 0.011767893451041045 0.011348513547901024 … 0.014476087195840908 0.02364311942474644], 0.0389, [0.10605520914187136, 0.09992562742912171, -0.14843493497180688, -0.05322598472545269, 0.12746584273127884, 0.3467801969777642, -0.032552830505324225, -0.054515187046808786, -0.09140617300232397, -0.13343594002000525, -0.1927607732594397, -0.1290652574111987, -0.01620911970895959], [1.1945864028660338, 1.1520141674858755, 1.1842729432657346, 1.6952654797364899, 1.7391506724730643, 1.7492585467618147, 1.3100065279696325, 1.3594224326415165, 1.2383042823206316, 1.5007541917289342, 1.4335572076293128, 1.4695014874837116, 0.5395559220282856], 0.10610026953971988)

Fill me in later.

In [12]:
solution = let
    solution = solve(model)
    solution

end

TypeError: TypeError: non-boolean (Int64) used in boolean context

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___